In [3]:
%matplotlib inline

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

import pandas as pd
from Utils import *

from algorithms import *
from new_utils import *

# Comparison of algorithms for ordinal embedding

In [11]:
#Create data|
dimensions= 3
number_of_points= 30

X = np.random.random((number_of_points, dimensions))
X = center_data(X)

In [27]:
n,d = X.shape
pulls = 20*int(number_of_points*dimensions*np.log(number_of_points))
print('Dimensions:{} \nNumber of points:{} \nPulls:{}'.format(dimensions,number_of_points, pulls))
# NOISE RUINS EVERYTHING, talk to blake about how to measure acc in this case
triplets, error = getTriplets(X, pulls, noise=False) 
    
# print(ste_loss(X, triplets,1))

Dimensions:3 
Number of points:30 
Pulls:6120


# Algorithms

In [ ]:
# from projected_gradient import *

# M, loss, proj_grad_loss_arr = computeEmbedding(M0,
#                 n,
#                  d,
#                  triplets,
#                  num_random_restarts=0,
#                  max_iter_GD=500,
#                  max_norm=1,
#                  epsilon=0.0001,
#                  accuracy=accuracy, 
#                  verbose=False)

# from crowd_kernel import *
# X, emp_loss_train, projection_free_loss_arr = computeEmbedding(n,d,triplets,alpha=1,
#                                     num_random_restarts=0,
#                                     epsilon=0.0001,
#                                     accuracy = accuracy,
#                                     max_iters=100,
#                                     verbose=False)


In [13]:
epsilons = np.linspace(0.005, 0.1,5)
epsilons

array([ 0.005  ,  0.02875,  0.0525 ,  0.07625,  0.1    ])

In [30]:
# Non Convex SGD
exp = 4
X0 = np.random.random((n,d))
stats_non_convex_single_exp = triplet_algorithms(ste_loss, 
                           triplets,
                           X0,                       
                           d,
                           'full_grad', 
                           50,
                           iters=5000,
                           epsilon = epsilons[exp],
                           proj=None,
                           debug=True
                          )

0 LOG ERROR 0.6166866633001885 Emp error 0.3503267973856209
1 LOG ERROR 0.6943461246015905 Emp error 0.3397058823529412
2 LOG ERROR 1.5368183753309221 Emp error 0.3720588235294118
3 LOG ERROR 12.253584807652675 Emp error 0.4091503267973856
Step size was too big, halving it. Iteration #:  4
4 LOG ERROR 19.65691624949381 Emp error 0.3633986928104575
Step size was too big, halving it. Iteration #:  5
5 LOG ERROR 1.0104078476913703 Emp error 0.18480392156862746
6 LOG ERROR 0.5387578971293439 Emp error 0.13480392156862744
7 LOG ERROR 0.4373370728897131 Emp error 0.12565359477124183
8 LOG ERROR 0.3853408517521151 Emp error 0.11274509803921569
9 LOG ERROR 0.3482375679232195 Emp error 0.10686274509803921
10 LOG ERROR 0.3146051128868386 Emp error 0.10196078431372549
11 LOG ERROR 0.2782845777215761 Emp error 0.09395424836601307
Accuracy reached in 11 iterations
Exiting


In [31]:
# CONVEX STE FULL GRAD Single experiment
M0 = np.random.randn(n,n)
stats_convex_single_exp = []
stats2 = triplet_algorithms(ste_loss_convex, 
                       triplets,
                       M0,                       
                       d,                            
                       'full_grad', 
                        50,
                       iters=5000,
                       epsilon =epsilons[exp],
                       proj=projected,
                       debug= True)
    


0 LOG ERROR 0.8417277463139345 Emp error 0.4571895424836601
1 LOG ERROR 0.7842213403895283 Emp error 0.4338235294117647
2 LOG ERROR 0.7332968807847291 Emp error 0.4106209150326797
3 LOG ERROR 0.6874122633245447 Emp error 0.3815359477124183
4 LOG ERROR 0.645589799892834 Emp error 0.35081699346405226
5 LOG ERROR 0.607271095291276 Emp error 0.3235294117647059
6 LOG ERROR 0.5721885451518046 Emp error 0.29215686274509806
7 LOG ERROR 0.5402351703103008 Emp error 0.263562091503268
8 LOG ERROR 0.5113414553792363 Emp error 0.23398692810457516
9 LOG ERROR 0.48538598590870097 Emp error 0.21601307189542485
10 LOG ERROR 0.4621637063821772 Emp error 0.1968954248366013
11 LOG ERROR 0.44140691043464286 Emp error 0.1769607843137255
12 LOG ERROR 0.42282622552662247 Emp error 0.15784313725490196
13 LOG ERROR 0.4061428703105191 Emp error 0.14166666666666666
14 LOG ERROR 0.39110461837762256 Emp error 0.12777777777777777
15 LOG ERROR 0.37749070694590203 Emp error 0.11568627450980393
16 LOG ERROR 0.365111858

In [ ]:
# FUll Experimental setup
stats_non_convex = []

for epsilon in epsilons:
    print('Epsilon', epsilon)
    X0 = np.random.random((n,d))
    stats1= triplet_algorithms(ste_loss, 
                       triplets,
                       X0,                       
                       d,
                       'full_grad', 
                       20,
                       iters=1000,
                       epsilon = epsilon,
                       proj=None,
                       debug=False)
    
    stats_non_convex.append(stats1)
    print()
    


In [ ]:
# Experiment number i.e which level of accuracy 
exp = 0

df = pd.DataFrame([
#         stats_convex[3]['emp'],
                   stats_non_convex[exp]['emp'],
#                    stats3['emp'],
#                     list(np.array(stats4['emp'])[epoch_inds])
                  ]).T

df.columns = [
#     'Proj Grad: Full GD (constant stepsize, shrink each epoch)',
              'Non convex: Full GD (constant stepsize, shrink each epoch)', 
#               'Proj Grad: SGD (constant stepsize)',
#               'Non Convex: SGD (constant stepsize, shrink each epoch)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Epochs', fontsize=22)
ax.set_title('Epsilon= {}'.format(epsilons[exp]), fontsize=22)
ax.legend(fontsize=18);

df = []
for i in stats_non_convex:
    df.append(sum(i['time_per_iter']))
    
df = pd.DataFrame(df, columns=['Non convex FULL GD'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');        

In [ ]:

# Non Convex SGD
exp = 2
stats4= triplet_algorithms(ste_loss, 
                           triplets,
                           X0,                       
                           d,
                           'sgd', 
                           0.2,
                           iters=5000,
                           epsilon = epsilons[exp],
                           proj=None,
                           debug=True
                          )

In [ ]:
# CONVEX STE FULL GRAD Full exp set up
M0 = np.random.randn(n,n)
stats_convex = []
for epsilon in epsilons:
    stats2 = triplet_algorithms(ste_loss_convex, 
                       triplets,
                       M0,                       
                       d,                            
                       'full_grad', 
                        600,
                       iters=5000,
                       epsilon =epsilon,
                       proj=projected,
                       debug= False)
    
    stats_convex.append(stats2)

In [ ]:
df = []
for i in stats_convex:
    df.append(sum(i['time_per_iter']))
    
df = pd.DataFrame(df, columns=['Non convex'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');        

In [ ]:
# stats3 = triplet_algorithms(ste_loss_convex, 
#                             triplets,
#                             M0,                       
#                             d,                            
#                             'sgd', 
#                             3,
#                             iters=5000,
#                             epsilon = 0.07625,
#                             proj=projected)

In [ ]:
# stats2['emp']

In [ ]:
epoch_inds = np.linspace(0, len(stats4['emp'])-1, stats4['epoch_count'], dtype=int)

df = pd.DataFrame([stats_convex[3]['emp'],
                   stats_non_convex[3]['emp'],
#                    stats3['emp'],
                    list(np.array(stats4['emp'])[epoch_inds])
                  ]).T
df.columns = ['Proj Grad: Full GD (constant stepsize, shrink each epoch)',
              'Non convex: Full GD (constant stepsize, shrink each epoch)', 
#               'Proj Grad: SGD (constant stepsize)',
              'Non Convex: SGD (constant stepsize, shrink each epoch)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Epochs', fontsize=22)
ax.set_title('Epsilon= {}'.format(0.07625), fontsize=22)
ax.legend(fontsize=18);

In [ ]:
df = pd.DataFrame([
#                 stats2['emp'],
#                 stats1['emp'],
                   stats3['emp'],
                   stats4['emp'],                   
                  ]
                 ).T
df.columns = [
#     'Proj Grad: Full GD (constant stepsize)',
#               'Non convex full gradient descent (constant stepsize)', 
              'Proj Grad: SGD (constant stepsize)',
              'Non Convex: SGD (constant stepsize)',              
             ]

ax = df.plot(figsize=(18,5), fontsize=18)
ax.set_ylabel('0-1 loss', fontsize=22)
ax.set_xlabel('Iterations of full gradient descent', fontsize=22)
ax.legend(fontsize=22);

In [ ]:
sum(stats1['time_per_iter'])

In [ ]:
sum(stats2['time_per_iter'])

In [ ]:
df = []
for i,j in zip(stats_non_convex,stats_convex):
    df.append([sum(i['time_per_iter']), len(j['emp'])*stats_convex[0]['avg_time_per_iter']])
    
df = pd.DataFrame(df, columns=['Non convex', 'Convex- with projection'], index=np.linspace(0.005, 0.1,5))
ax = df.plot(figsize=(18,8), fontsize=18, kind='bar')
ax.set_ylabel('Time(sec)', fontsize=22)
ax.set_xlabel('Error', fontsize=22)
ax.legend(fontsize=22, loc='upper right');    

In [ ]:
# for i in range(5):
#     print(len(stats_convex[i]['emp']))
#     print(stats_convex[i]['time_per_iter'])
#     print()

In [ ]:
epsilons